In [1]:
%load_ext autoreload
%autoreload 2
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import processing_netcdf as pcdf
import geopandas as gpd
from geopandas import GeoDataFrame
import shapely.geometry 
import numpy as np
from shapely import geometry as gmty
from geofeather import to_geofeather, from_geofeather
import glob
import os

In [2]:
#path to data
data = "/home/mlopez/EXEC/Observed data/NRCAN_obs_prcptot_annual.nc"

# Create DF of observed data

In [3]:
dfobstpr= pcdf.load_as_df(data)

In [4]:
dfobstpr.reset_index()

prcptot
lat       lon        time                   
83.125000 -74.541672 1950-01-01   111.560005
                     1951-01-01   138.380020
                     1952-01-01   105.830017
                     1953-01-01   157.129959
                     1954-01-01   122.549988
...                                      ...
41.708332 -82.208336 2009-01-01   792.690430
                     2010-01-01   793.760071
                     2011-01-01  1290.789917
                     2012-01-01   684.479980
                     2013-01-01   971.789856

[16464832 rows x 1 columns]

# Define periods of time: 

In [6]:
year_groups = {y:0 for y in range(1980,2011)}
#Get the mean of every period
dfC2 = dfobstpr.reset_index()
dfp = dfC2.groupby([dfC2.time.dt.year.map(year_groups), "lat","lon"]).mean()

In [7]:
dfp.reset_index()

,time,lat,lon,prcptot
0,0.0,41.708332,-82.791672,844.777710
1,0.0,41.708332,-82.708336,847.000671
2,0.0,41.708332,-82.625000,847.128662
3,0.0,41.708332,-82.541672,847.252258
4,0.0,41.708332,-82.458336,846.754211
...,...,...,...,...
257258,0.0,83.125000,-69.958336,127.883537
257259,0.0,83.125000,-69.875000,125.947083
257260,0.0,83.125000,-69.791672,126.917740
257261,0.0,83.125000,-69.708336,126.662903


# Historic Period

In [8]:
df_h = dfp.query("time==0.0")
df_h.reset_index()

,time,lat,lon,prcptot
0,0.0,41.708332,-82.791672,844.777710
1,0.0,41.708332,-82.708336,847.000671
2,0.0,41.708332,-82.625000,847.128662
3,0.0,41.708332,-82.541672,847.252258
4,0.0,41.708332,-82.458336,846.754211
...,...,...,...,...
257258,0.0,83.125000,-69.958336,127.883537
257259,0.0,83.125000,-69.875000,125.947083
257260,0.0,83.125000,-69.791672,126.917740
257261,0.0,83.125000,-69.708336,126.662903


## Import dataframe with polygons

In [9]:
dfpolyshape = from_geofeather('Grid-TerritoiresGuides.feather')

/home/mlopez/EXEC/anaconda3/envs/python37-2/lib/python3.8/site-packages/geofeather/core.py:85: UserWarning: Grid-TerritoiresGuides.feather.crs coordinate reference system file is missing. No crs will be set for this GeoDataFrame.
  warnings.warn(


In [10]:
dfpolyshape

,lat,lon,TER_GUIDE,geometry
0,52.957191,-67.712730,6opqr,"POLYGON ((-67.67107 52.93424, -67.67107 52.915..."
1,52.957191,-67.629402,6opqr,"POLYGON ((-67.67107 52.91553, -67.67107 52.934..."
2,52.957191,-67.546066,6opqr,"POLYGON ((-67.56730 52.91553, -67.57974 52.915..."
3,52.873859,-67.796059,6opqr,"POLYGON ((-67.75439 52.89329, -67.75439 52.832..."
4,52.873859,-67.712730,6opqr,"POLYGON ((-67.75439 52.83219, -67.75439 52.893..."
...,...,...,...,...
33193,45.041668,-72.208336,2c,"POLYGON ((-72.25000 45.00450, -72.25000 45.083..."
33194,45.041668,-72.125000,2c,"POLYGON ((-72.16666 45.00508, -72.16666 45.083..."
33195,45.041668,-72.041672,2c,"POLYGON ((-72.08334 45.00564, -72.08334 45.083..."
33196,45.041668,-71.958336,2c,"POLYGON ((-72.00000 45.00667, -72.00000 45.083..."


## Merge data with mask 

In [11]:
dfprobsTG = pd.merge(df_h, dfpolyshape, on=["lat","lon"])

In [23]:
dfprobsTG

,lat,lon,prcptot,TER_GUIDE,geometry
0,44.958332,-74.625000,976.803528,1a,"POLYGON ((-74.66365 45.00000, -74.58334 45.000..."
1,44.958332,-74.541672,974.891296,1a,"POLYGON ((-74.58334 44.99872, -74.58334 45.000..."
2,44.958332,-74.458336,976.003784,1a,"POLYGON ((-74.50000 44.99779, -74.50000 45.000..."
3,44.958332,-74.375000,978.627075,1a,"POLYGON ((-74.41666 44.99517, -74.41666 45.000..."
4,44.958332,-74.291672,984.361572,1a,"POLYGON ((-74.33334 44.99213, -74.33334 45.000..."
...,...,...,...,...,...
16559,52.875000,-67.625000,861.823914,6opqr,"POLYGON ((-67.66666 52.83333, -67.66666 52.916..."
16560,52.875000,-67.541672,860.633850,6opqr,"POLYGON ((-67.58334 52.83333, -67.58334 52.911..."
16561,52.958332,-67.708336,861.630310,6opqr,"POLYGON ((-67.66667 52.93419, -67.66667 52.916..."
16562,52.958332,-67.625000,857.484863,6opqr,"POLYGON ((-67.66666 52.91667, -67.66666 52.934..."


In [13]:
dfprobsTG.TER_GUIDE.value_counts()

6cdefg    2361
6opqr     1609
6ab       1416
5bcd      1237
4bc       1048
6hi       1026
6j         949
6mn        925
3ab        652
5a         550
6kl        512
5ef        399
4f         394
2b         388
3d         365
3c         355
4de        355
5g         350
5hi        341
1a         340
4gh        301
5jk        245
2c         165
2a         142
4a         139
Name: TER_GUIDE, dtype: int64

# ---- Create GeoJson file ----

In [24]:
geometry = dfprobsTG["geometry"]

In [25]:
crs = {'init': "epsg:4326"}
gdf = GeoDataFrame(dfprobsTG, crs=crs, geometry=geometry)

In [26]:
gdf.to_file("TG_hist_prcptot_annual_rcp45_50.json", driver="GeoJSON")

# Create GeoJson for each territory

In [17]:
listTG = []

for tg in dfprobsTG['TER_GUIDE'].unique().tolist():
    df = dfprobsTG[dfprobsTG.TER_GUIDE == tg]
    print (tg)
    listTG.append(df)

listTG

1a
2c
3d
2b
2a
3c
3ab
4bc
4de
5ef
4f
4a
5jk
5bcd
5a
4gh
5hi
5g
6cdefg
6hi
6ab
6j
6mn
6kl
6opqr


[            lat        lon     prcptot TER_GUIDE  \
 0     44.958332 -74.625000  976.803528        1a   
 1     44.958332 -74.541672  974.891296        1a   
 2     44.958332 -74.458336  976.003784        1a   
 3     44.958332 -74.375000  978.627075        1a   
 4     44.958332 -74.291672  984.361572        1a   
 ...         ...        ...         ...       ...   
 1234  46.208332 -72.625000  984.718750        1a   
 1342  46.291668 -72.875000  987.298401        1a   
 1344  46.291668 -72.791672  988.051331        1a   
 1346  46.291668 -72.708336  992.107727        1a   
 1348  46.291668 -72.625000  997.760071        1a   
 
                                                geometry  
 0     POLYGON ((-74.66365 45.00000, -74.58334 45.000...  
 1     POLYGON ((-74.58334 44.99872, -74.58334 45.000...  
 2     POLYGON ((-74.50000 44.99779, -74.50000 45.000...  
 3     POLYGON ((-74.41666 44.99517, -74.41666 45.000...  
 4     POLYGON ((-74.33334 44.99213, -74.33334 45.000...  
 ...    

In [22]:
for tg in listTG:
    geometry = tg["geometry"]
    crs = {'init': "epsg:4326"}
    gdf = GeoDataFrame(tg, crs=crs, geometry=geometry)
    #print (gdf['TER_GUIDE'].iloc[0])
    gdf.to_file(gdf['TER_GUIDE'].iloc[0]+"_hist_prcptot_annual_rcp45_50.json", driver="GeoJSON")